In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from cmap import ncl_colormap 
from mpl_toolkits.basemap import Basemap, addcyclic
from my_tools import file_dic, plot_dic
from glob import glob
import matplotlib 
matplotlib.use('agg')
%matplotlib qt

In [2]:
path = '/home/onno/Thesis'

In [3]:
#load RWP property files
rean_env_gen = xr.open_dataset(path +'era51_mars_env_wledit2000-10000_latavg_v300_79-19_6hourly_anom_from_smoothed04_clim_smoothed.nc').sel(latitude=slice(90,0)).squeeze()
rean_v300_gen = xr.open_dataset(path + 'era51_mars_v300_wledit2000-10000_latavg_79-19_6hourly.nc').sel(latitude=slice(90,0)).squeeze()
rean_env_gen =  rean_env_gen.assign_coords(longitude=(((rean_env_gen.longitude + 180) % 360) - 180)).sortby('longitude')
rean_v300_gen =  rean_v300_gen.assign_coords(longitude=(((rean_v300_gen.longitude + 180) % 360) - 180)).sortby('longitude')

rean_env_gen=rean_env_gen.rename({'longitude':'lon'});rean_env_gen=rean_env_gen.rename({'latitude':'lat'})
rean_v300_gen=rean_v300_gen.rename({'longitude':'lon'});rean_v300_gen=rean_v300_gen.rename({'latitude':'lat'})
# rean_env_gen = rean_env_gen.sel(time=[bool(i) for i in rean_env_gen.time.dt.hour%24==0])
# rean_v300_gen = rean_v300_gen.sel(time=[bool(i) for i in rean_v300_gen.time.dt.hour%24==0])

ERA5RF_env_gen = xr.open_dataset(path+'era5rf_env_wledit2000-10000_latavg_v300_0-240h_12hourly_2x2nh_jan79-dec19.nc',decode_times=False).squeeze()
GFS_env_gen = xr.open_dataset(path+'gefsrf2_env_wledit2000-10000_latavg_v300_control0-252h_6hourly_2x2_dec84-nov19.nc',decode_times=False).squeeze()
ERA5RF_v300_gen = xr.open_dataset(path+'era5rf_v300_wledit2000-10000_latavg_0-240h_12hourly_2x2nh_jan79-dec19.nc',decode_times=False).squeeze()
GFS_v300_gen = xr.open_dataset(path+'gefsrf2_v300_control0-252h_6hourly_2x2_dec84-nov19.nc',decode_times=False).squeeze()

GFS_init_time = pd.Timestamp('1800-01-01')
ERA5RF_init_time = pd.Timestamp('1900-01-01')
GFS_env_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_env_gen.time.values]
GFS_v300_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_v300_gen.time.values]
ERA5RF_env_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_env_gen.time.values]
ERA5RF_v300_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_v300_gen.time.values]

# GFS_env_gen = GFS_env_gen.rename({'lon':'longitude','lat':'latitude'}); GFS_v300_gen = GFS_v300_gen.rename({'lon':'longitude','lat':'latitude'})
# ERA5RF_env_gen = ERA5RF_env_gen.rename({'lon':'longitude','lat':'latitude'}); ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'lon':'longitude','lat':'latitude'})
GFS_env_gen = GFS_env_gen.assign_coords(lon=(((GFS_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
GFS_v300_gen = GFS_v300_gen.assign_coords(lon=(((GFS_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
ERA5RF_env_gen = ERA5RF_env_gen.assign_coords(lon=(((ERA5RF_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
ERA5RF_v300_gen = ERA5RF_v300_gen.assign_coords(lon=(((ERA5RF_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# GFS_env_gen = GFS_env_gen.sel(lead=GFS_env_gen.lead.values[[bool(i) for i in GFS_env_gen.lead%24==0]])
# GFS_v300_gen = GFS_v300_gen.sel(lead=GFS_v300_gen.lead.values[[bool(i) for i in GFS_v300_gen.lead%24==0]])
# ERA5RF_env_gen = ERA5RF_env_gen.sel(lead=ERA5RF_env_gen.lead.values[[bool(i) for i in ERA5RF_env_gen.lead%24==0]])
# # ERA5RF_v300_gen = ERA5RF_v300_gen.sel(lead=ERA5RF_v300_gen.lead.values[[bool(i) for i in ERA5RF_v300_gen.lead%24==0]])

# GFS_env_gen['lead'] = GFS_env_gen.lead.values//24
# GFS_v300_gen['lead'] = GFS_v300_gen.lead.values//24
# ERA5RF_env_gen['lead'] = ERA5RF_env_gen.lead.values//24
# # ERA5RF_v300_gen['lead'] = ERA5RF_v300_gen.lead.values//24

# GFS_env_gen = GFS_env_gen.rename({'latitude':'lat','longitude':'lon'})
# GFS_v300_gen = GFS_v300_gen.rename({'latitude':'lat','longitude':'lon'})
# ERA5RF_env_gen = ERA5RF_env_gen.rename({'latitude':'lat','longitude':'lon'})
# # ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'latitude':'lat','longitude':'lon'})





In [9]:
coordinatez = [
# (54,46,6,14),#Germany
(44,36,352,360), #Spain
# (54,46,26,34), #Ukraine
(58,50,352,360)] #UK
# (42,34,28,36), #Turkey
# (68,60,22,30), #Finland
# (66,58,6,14), #Norway/Sweden
# (60,52,46,54)] #Russia Samara/Kazan region #Germany
file = '/GFS_T850/rank_forecast/{}_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'
eventz = ['persistent_hw','persistent_cw','short_hw','short_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave','Short Heatwave','Short Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(24,61,2),
                     'persistent_cw': np.arange(24,61,2),
                     'short_hw':np.arange(24,61,2),
                     'short_cw':np.arange(24,61,2)},
               'JJA':{'persistent_hw':np.arange(16,61,2),
                     'persistent_cw': np.arange(16,61,2),
                     'short_hw':np.arange(16,61,2),
                     'short_cw':np.arange(16,61,2)}}

lead_day = 5
for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    for i,event in enumerate(eventz):
#         datez_event = np.load(path + 'GFS_T850/{}_lon_{}_{}_lat_{}_{}.npy'.format(event,lon_0,lon_1-2,lat_0-2,lat_1))
        for model in modelz:
            for season in seasonz:
                df_good = pd.read_csv(path+file.format('good',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_bad = pd.read_csv(path+file.format('bad',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_good.index = pd.to_datetime(df_good.index)
                df_bad.index = pd.to_datetime(df_bad.index)
                reanz_env_good = []
                reanz_v300_good = []
                lat_max = lat_0 + 6
                lat_min = lat_1 - 8
                for date in df_good.index:
                    time_begin = date - pd.Timedelta(10,'days')
                    time_end = date + pd.Timedelta(10,'days')
                    rean_env_good_sub = rean_env_gen.sel(lat=slice(lat_max,lat_min),
                                                   time=slice(time_begin,time_end))
                    rean_v300_good_sub = rean_v300_gen.sel(lat=slice(lat_max,lat_min),
                                                   time=slice(time_begin,time_end))
                    rean_env_good_sub = rean_env_good_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_good_sub.time)))
                    rean_v300_good_sub = rean_v300_good_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_good_sub.time)))

                    reanz_env_good.append(rean_env_good_sub)
                    reanz_v300_good.append(rean_v300_good_sub)
                rean_env_good = xr.concat(reanz_env_good,dim='time').groupby('time').mean().mean('lat')
                rean_v300_good = xr.concat(reanz_v300_good,dim='time').groupby('time').mean().mean('lat')
                rean_env_good.attrs['lon_0']=lon_0
                rean_v300_good.attrs['lon_0']=lon_0
                rean_env_good.to_netcdf(path+'fcst_composite/hovmoller/reanalysis/rawdata/'+\
                                    'good_forecast_envelope_hovmoller_{}_lon_{}_{}_lat_{}_{}_{}_{}.nc'.format(event,lon_0,lon_1,
                                                                                                       lat_0,lat_1,season,model))
                rean_v300_good.to_netcdf(path+'fcst_composite/hovmoller/reanalysis/rawdata/'+\
                                    'good_forecast_v_wind_hovmoller_{}_lon_{}_{}_lat_{}_{}_{}_{}.nc'.format(event,lon_0,lon_1,
                                                                                                       lat_0,lat_1,season,model))
                reanz_env_bad = []
                reanz_v300_bad = []
                for date in df_bad.index:
                    time_begin = date - pd.Timedelta(10,'days')
                    time_end = date + pd.Timedelta(10,'days')
                    rean_env_bad_sub = rean_env_gen.sel(lat=slice(lat_max,lat_min),
                                                   time=slice(time_begin,time_end))
                    rean_v300_bad_sub = rean_v300_gen.sel(lat=slice(lat_max,lat_min),
                                                   time=slice(time_begin,time_end))
                    rean_env_bad_sub = rean_env_bad_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_bad_sub.time)))
                    rean_v300_bad_sub = rean_v300_bad_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_bad_sub.time)))

                    reanz_env_bad.append(rean_env_bad_sub)
                    reanz_v300_bad.append(rean_v300_bad_sub)
                rean_env_bad = xr.concat(reanz_env_bad,dim='time').groupby('time').mean().mean('lat')
                rean_v300_bad = xr.concat(reanz_v300_bad,dim='time').groupby('time').mean().mean('lat')
                if lon_0>180:
                    lon_0_plot = lon_0 - 360
                    lon_1_plot = lon_1 - 360
                rean_env_bad.attrs['lon_0']=lon_0_plot
                rean_v300_bad.attrs['lon_0']=lon_0_plot
                rean_env_bad.to_netcdf(path+'fcst_composite/hovmoller/reanalysis/rawdata/'+\
                                    'bad_forecast_envelope_hovmoller_{}_lon_{}_{}_lat_{}_{}_{}_{}.nc'.format(event,lon_0,lon_1,
                                                                                                       lat_0,lat_1,season,model))
                rean_v300_bad.to_netcdf(path+'fcst_composite/hovmoller/reanalysis/rawdata/'+\
                                    'bad_forecast_v_wind_hovmoller_{}_lon_{}_{}_lat_{}_{}_{}_{}.nc'.format(event,lon_0,lon_1,
                                                                                                       lat_0,lat_1,season,model))
                fig,axz=plt.subplots(2,1,figsize=(16,9))
                ax1 =axz.flat[0];ax2=axz.flat[1]
                lonz = rean_env_good.lon.values
                dayz = rean_env_good.time.values

                clr_levels = np.concatenate((np.arange(-30,-5,2),np.arange(6,31,2)))
                contour_levels = contour_dic[season][event]
                im = ax1.contourf(lonz,dayz,rean_v300_good.v.values,cmap='bwr',levels=clr_levels,extend='both')
                ax1.contour(lonz,dayz,rean_env_good.v.values,levels=contour_levels)
                ax1.set_title('Good Forecasts N = {}'.format(len(df_good.index)))
                ax1.axvline(x=lon_0_plot,color='grey');ax1.axvline(x=lon_1_plot,color='grey')
                ax1.axhline(y=0,color='grey')
                ax1.set_ylabel('Days before/after onset event')


                ax2.contourf(lonz,dayz,rean_v300_bad.v.values,cmap='bwr',levels=clr_levels,extend='both')
                ax2.contour(lonz,dayz,rean_env_bad.v.values,levels=contour_levels)

                ax2.set_title('Bad Forecasts N = {}'.format(len(df_bad.index)))
                ax2.axvline(x=lon_0_plot,color='grey');ax2.axvline(x=lon_1_plot,color='grey')
                ax2.axhline(y=0,color='grey')
                ax2.set_ylabel('Days before/after onset event')
                ax2.set_xlabel('Longitude [°]')
                fig.suptitle('{} {} {}'.format(event_titlez[i],season,model))

                fig.subplots_adjust(right=0.8)
                cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
                cbar = fig.colorbar(im, cax=cbar_ax)
                cbar.ax.get_yaxis().set_ticks([-30,-20,-10,10,20,30])
                cbar.ax.set_yticklabels([-30,-20,-10,10,20,30])
                cbar.ax.set_ylabel('v (m/s)')

                fig.savefig(path+'fcst_composite/hovmoller/reanalysis/hovmoller_reanalysis_{}_{}_lon_{}_{}_lat_{}_{}_{}.png'.format(
                                                                                                                model,event,
                                                                                                                lon_0,lon_1,
                                                                                                                lat_0,lat_1,
                                                                                                                season))
                plt.close(fig)







/home/onno/miniconda3/envs/thesis/lib/python3.6/site-packages/ipykernel_launcher.py:106: UserWarning: No contour levels were found within the data range.


Hovmoller Forecast vs Reanalysisn for meridional wind

In [64]:
coordinatez = [
(54,46,6,14)] #Germany
file = '/GFS_T850/rank_forecast/{}_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'
eventz = ['persistent_hw','persistent_cw','short_hw','short_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave','Short Heatwave','Short Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
modelz_data = [GFS_v300_gen,ERA5RF_v300_gen]
contour_dic = {'DJF':{'persistent_hw':np.arange(24,61,2),
                     'persistent_cw': np.arange(24,61,2),
                     'short_hw':np.arange(24,61,2),
                     'short_cw':np.arange(24,61,2)},
               'JJA':{'persistent_hw':np.arange(16,61,2),
                     'persistent_cw': np.arange(16,61,2),
                     'short_hw':np.arange(16,61,2),
                     'short_cw':np.arange(16,61,2)}}

lead_day = 3
for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    for i,event in enumerate(eventz):
        datez_event = np.load(path + 'GFS_T850/{}_lon_{}_{}_lat_{}_{}.npy'.format(event,lon_0,lon_1-2,lat_0-2,lat_1))
        for j,model in enumerate(modelz):
            fcst_data = modelz_data[j]
            for season in seasonz:
                df_good = pd.read_csv(path+file.format('good',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_bad = pd.read_csv(path+file.format('bad',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_good.index = pd.to_datetime(df_good.index)
                df_bad.index = pd.to_datetime(df_bad.index)
                fcstz_v300_good = []
                reanz_v300_good = []
                for date in df_good.index:
                    time_begin = date - pd.Timedelta(lead_day,'days')
                    time_end = time_begin + pd.Timedelta(10,'days')
#                     rean_env_good_sub = rean_env_gen.sel(lat=slice(70,40),
#                                                    time=slice(time_begin,time_end))
                    rean_v300_good_sub = rean_v300_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    if model=='ERA5RF':
                        rean_v300_good_sub = rean_v300_good_sub.sel(time = rean_v300_good_sub.time.values[[bool(i) for i in rean_v300_good_sub.time.dt.hour%12==0]])
                    rean_v300_good_sub = rean_v300_good_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_v300_good_sub.time)))
#                     rean_v300_good_sub = rean_v300_good_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_good_sub.time)))
                    fcst_v300_good_sub = fcst_data.sel(lat=slice(70,40),time=time_begin,lead=slice(0,240))
                    fcst_v300_good_sub = fcst_v300_good_sub.assign_coords(lead=np.linspace(-lead_day,-lead_day+10,len(rean_v300_good_sub.time)))

#                     reanz_env_good.append(rean_env_good_sub)
                    reanz_v300_good.append(rean_v300_good_sub)
                    fcstz_v300_good.append(fcst_v300_good_sub)
#                 rean_env_good = xr.concat(reanz_env_good,dim='time').groupby('time').mean().mean('lat')
                rean_v300_good = xr.concat(reanz_v300_good,dim='time').groupby('time').mean().mean('lat')
                fcst_v300_good = xr.concat(fcstz_v300_good,dim='lead').groupby('lead').mean().mean('lat')
                rean_env_good.to_csv(path+'fcst_composite/hovmoller/reanalysis/rawdata/'+\
                                    'good_forecast_envelope_hovmoller_{}_lon_{}_{}_lat_{}_{}_{}_{}'.format(event,lon_0,lon_1,
                                                                                                       lat_0,lat_1,season,model))
                rean_env_good.to_csv(path+'fcst_composite/hovmoller/reanalysis/rawdata/'+\
                                    'good_forecast_v_wind_hovmoller_{}_lon_{}_{}_lat_{}_{}_{}_{}'.format(event,lon_0,lon_1,
                                                                                                       lat_0,lat_1,season,model))
                fcstz_v300_bad = []
                reanz_v300_bad = []
                for date in df_bad.index:
                    time_begin = date - pd.Timedelta(lead_day,'days')
                    time_end = time_begin + pd.Timedelta(10,'days')
#                     rean_env_bad_sub = rean_env_gen.sel(lat=slice(70,40),
#                                                    time=slice(time_begin,time_end))
                    rean_v300_bad_sub = rean_v300_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    if model=='ERA5RF':
                        rean_v300_bad_sub = rean_v300_bad_sub.sel(time = rean_v300_bad_sub.time.values[[bool(i) for i in rean_v300_bad_sub.time.dt.hour%12==0]])
                    rean_v300_bad_sub = rean_v300_bad_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_v300_bad_sub.time)))
#                     rean_v300_bad_sub = rean_v300_bad_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_bad_sub.time)))
                    fcst_v300_bad_sub = fcst_data.sel(lat=slice(70,40),time=time_begin,lead=slice(0,240))
                    fcst_v300_bad_sub = fcst_v300_bad_sub.assign_coords(lead=np.linspace(-lead_day,-lead_day+10,len(rean_v300_bad_sub.time)))

#                     reanz_env_bad.append(rean_env_bad_sub)
                    reanz_v300_bad.append(rean_v300_bad_sub)
                    fcstz_v300_bad.append(fcst_v300_bad_sub)
#                 rean_env_bad = xr.concat(reanz_env_bad,dim='time').groupby('time').mean().mean('lat')
                rean_v300_bad = xr.concat(reanz_v300_bad,dim='time').groupby('time').mean().mean('lat')
                fcst_v300_bad = xr.concat(fcstz_v300_bad,dim='lead').groupby('lead').mean().mean('lat')
                fig,axz=plt.subplots(2,1,figsize=(16,9))
                ax1 =axz.flat[0];ax2=axz.flat[1]
                lonz = rean_v300_bad.lon.values
                dayz = rean_v300_bad.time.values

                clr_levels = np.concatenate((np.arange(-30,-5,2),np.arange(6,31,2)))
                contour_levels = contour_dic[season][event]
                im = ax1.contourf(lonz,dayz,rean_v300_good.v.values,cmap='bwr',levels=clr_levels,extend='both')
                ax1.contour(lonz,dayz,fcst_v300_good.v.values,levels=clr_levels)
                ax1.set_title('Good Forecasts N = {}'.format(len(df_good.index)))
                ax1.axvline(x=6,color='grey');ax1.axvline(x=14,color='grey')
                ax1.axhline(y=0,color='grey')
                ax1.set_ylabel('Days before/after onset event')


                ax2.contourf(lonz,dayz,rean_v300_bad.v.values,cmap='bwr',levels=clr_levels,extend='both')
                ax2.contour(lonz,dayz,fcst_v300_bad.v.values,levels=clr_levels)

                ax2.set_title('Bad Forecasts N = {}'.format(len(df_bad.index)))
                ax2.axvline(x=6,color='grey');ax2.axvline(x=14,color='grey')
                ax2.axhline(y=0,color='grey')
                ax2.set_ylabel('Days before/after onset event')
                ax2.set_xlabel('Longitude [°]')
                fig.suptitle('{} {} {}'.format(event_titlez[i],season,model))

                fig.subplots_adjust(right=0.8)
                cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
                cbar = fig.colorbar(im, cax=cbar_ax)
                cbar.ax.get_yaxis().set_ticks([-30,-20,-10,10,20,30])
                cbar.ax.set_yticklabels([-30,-20,-10,10,20,30])
                cbar.ax.set_ylabel('v (m/s)')

                fig.savefig(path+'fcst_composite/hovmoller/forecast/hovmoller_forecast_{}_{}_lon_{}_{}_lat_{}_{}_{}_lead_day_{}.png'.format(
                                                                                                                model,event,
                                                                                                                lon_0,lon_1,
                                                                                                                lat_0,lat_1,
                                                                                                                season,lead_day))
                plt.close(fig)







Hovmoller Forecast vs Reanalysisn for RWP envelope

In [56]:
coordinatez = [
(54,46,6,14)] #Germany
file = '/GFS_T850/rank_forecast/{}_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'
eventz = ['persistent_hw','persistent_cw','short_hw','short_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave','Short Heatwave','Short Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
modelz_data = [GFS_env_gen,ERA5RF_env_gen]
contour_dic = {'DJF':{'persistent_hw':np.arange(24,61,2),
                     'persistent_cw': np.arange(24,61,2),
                     'short_hw':np.arange(24,61,2),
                     'short_cw':np.arange(24,61,2)},
               'JJA':{'persistent_hw':np.arange(16,61,2),
                     'persistent_cw': np.arange(16,61,2),
                     'short_hw':np.arange(16,61,2),
                     'short_cw':np.arange(16,61,2)}}

lead_day = 5
for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    for i,event in enumerate(eventz):
        datez_event = np.load(path + 'GFS_T850/{}_lon_{}_{}_lat_{}_{}.npy'.format(event,lon_0,lon_1-2,lat_0-2,lat_1))
        for j,model in enumerate(modelz):
            fcst_data = modelz_data[j]
            for season in seasonz:
                df_good = pd.read_csv(path+file.format('good',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_bad = pd.read_csv(path+file.format('bad',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_good.index = pd.to_datetime(df_good.index)
                df_bad.index = pd.to_datetime(df_bad.index)
                fcstz_env_good = []
                reanz_env_good = []
                for date in df_good.index:
                    time_begin = date - pd.Timedelta(lead_day,'days')
                    time_end = time_begin + pd.Timedelta(10,'days')
                    rean_env_good_sub = rean_env_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    if model=='ERA5RF':
                        rean_env_good_sub = rean_env_good_sub.sel(time = rean_env_good_sub.time.values[[bool(i) for i in rean_env_good_sub.time.dt.hour%12==0]])
                    rean_env_good_sub = rean_env_good_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_env_good_sub.time)))
                    fcst_env_good_sub = fcst_data.sel(lat=slice(70,40),time=time_begin,lead=slice(0,240))
                    fcst_env_good_sub = fcst_env_good_sub.assign_coords(lead=np.linspace(-lead_day,-lead_day+10,len(rean_env_good_sub.time)))

                    reanz_env_good.append(rean_env_good_sub)
                    fcstz_env_good.append(fcst_env_good_sub)
                rean_env_good = xr.concat(reanz_env_good,dim='time').groupby('time').mean().mean('lat')
                fcst_env_good = xr.concat(fcstz_env_good,dim='lead').groupby('lead').mean().mean('lat')
                fcstz_env_bad = []
                reanz_env_bad = []
                for date in df_bad.index:
                    time_begin = date - pd.Timedelta(lead_day,'days')
                    time_end = time_begin + pd.Timedelta(10,'days')
                    rean_env_bad_sub = rean_env_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    if model=='ERA5RF':
                        rean_env_bad_sub = rean_env_bad_sub.sel(time = rean_env_bad_sub.time.values[[bool(i) for i in rean_env_bad_sub.time.dt.hour%12==0]])
                    rean_env_bad_sub = rean_env_bad_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_env_bad_sub.time)))
                    fcst_env_bad_sub = fcst_data.sel(lat=slice(70,40),time=time_begin,lead=slice(0,240))
                    fcst_env_bad_sub = fcst_env_bad_sub.assign_coords(lead=np.linspace(-lead_day,-lead_day+10,len(rean_env_bad_sub.time)))

                    reanz_env_bad.append(rean_env_bad_sub)
                    fcstz_env_bad.append(fcst_env_bad_sub)
                rean_env_bad = xr.concat(reanz_env_bad,dim='time').groupby('time').mean().mean('lat')
                fcst_env_bad = xr.concat(fcstz_env_bad,dim='lead').groupby('lead').mean().mean('lat')
                fig,axz=plt.subplots(2,1,figsize=(16,9))
                ax1 =axz.flat[0];ax2=axz.flat[1]
                lonz = rean_env_bad.lon.values
                dayz = rean_env_bad.time.values

                clr_levels = np.concatenate((np.arange(-30,-5,2),np.arange(6,31,2)))
                contour_levels = contour_dic[season][event]
                im = ax1.contourf(lonz,dayz,rean_env_good.v.values,cmap='ocean',levels=contour_levels)
                ax1.contour(lonz,dayz,fcst_env_good.v.values,levels=contour_levels)
                ax1.set_title('Good Forecasts N = {}'.format(len(df_good.index)))
                ax1.axvline(x=6,color='grey');ax1.axvline(x=14,color='grey')
                ax1.axhline(y=0,color='grey')
                ax1.set_ylabel('Days before/after onset event')


                ax2.contourf(lonz,dayz,rean_env_bad.v.values,cmap='ocean',levels=contour_levels)
                ax2.contour(lonz,dayz,fcst_env_bad.v.values,levels=contour_levels)

                ax2.set_title('Bad Forecasts N = {}'.format(len(df_bad.index)))
                ax2.axvline(x=6,color='grey');ax2.axvline(x=14,color='grey')
                ax2.axhline(y=0,color='grey')
                ax2.set_ylabel('Days before/after onset event')
                ax2.set_xlabel('Longitude [°]')
                fig.suptitle('{} {} {}'.format(event_titlez[i],season,model))

                fig.subplots_adjust(right=0.8)
                cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
                cbar = fig.colorbar(im, cax=cbar_ax)
#                 cbar.ax.get_yaxis().set_ticks([-30,-20,-10,10,20,30])
#                 cbar.ax.set_yticklabels([-30,-20,-10,10,20,30])
                cbar.ax.set_ylabel('v (m/s)')

                fig.savefig(path+'fcst_composite/hovmoller/forecast/hovmoller_envelope_forecast_{}_{}_lon_{}_{}_lat_{}_{}_{}_lead_day_{}.png'.format(
                                                                                                                model,event,
                                                                                                                lon_0,lon_1,
                                                                                                                lat_0,lat_1,
                                                                                                                season,lead_day))
                plt.close(fig)







FileNotFoundError: [Errno 2] No such file or directory: '/media/onno/Algemeen/Thesis/fcst_composite/hovmoller/reanalysis/GFS_T850/persistent_hw_lon_6_12_lat_52_46.npy'

Combined Hovmöllers (for all regions for every type of event)

In [12]:
# coordinatez = [
# (54,46,6,14),#Germany
# (44,36,352,360), #Spain
# (54,46,26,34), #Ukraine
# (58,50,352,360), #UK
# (42,34,28,36), #Turkey
# (68,60,22,30), #Finland
# (66,58,6,14), #Norway/Sweden
# (60,52,46,54)] #Russia Samara/Kazan region #Germany
path = '/media/onno/Algemeen/Thesis/fcst_composite/hovmoller/reanalysis/'
file = 'rawdata/{}_forecast_{}_hovmoller_{}_lon_*_*_lat_*_*_{}_{}.nc'
eventz = ['persistent_hw','persistent_cw','short_hw','short_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave','Short Heatwave','Short Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(20,35,1),
                     'persistent_cw': np.arange(20,35,2),
                     'short_hw':np.arange(20,35,1),
                     'short_cw':np.arange(20,35,1)},
               'JJA':{'persistent_hw':np.arange(15,25,1),
                     'persistent_cw': np.arange(15,25,1),
                     'short_hw':np.arange(15,25,1),
                     'short_cw':np.arange(15,25,1)}}
lead_day = 5
# for lat_0,lat_1,lon_0,lon_1 in coordinatez:
for i,event in enumerate(eventz):
    for model in modelz:
        for season in seasonz:
            filez_good_env = glob(path+file.format('good','envelope',event,season,model))
            filez_bad_env = glob(path+file.format('bad','envelope',event,season,model))
            filez_good_v300 = glob(path+file.format('good','v_wind',event,season,model))
            filez_bad_v300 = glob(path+file.format('bad','v_wind',event,season,model))
            filez_all = [filez_good_env,filez_bad_env,filez_good_v300,filez_bad_v300]
            filez_all_list = []
            for filez in filez_all:
                filez_list = []
                for file_ in filez:
                    ds = xr.open_dataset(file_)
                    ds_new = ds.assign_coords(lon=(((ds.lon-ds.attrs['lon_0']+180)%360)-180)).sortby('lon')
                    filez_list.append(ds_new)
                filez_all_list.append(filez_list)
            ds_good_env = xr.concat(filez_all_list[0],dim='region').mean(dim='region')
            ds_bad_env = xr.concat(filez_all_list[1],dim='region').mean(dim='region')
            ds_good_v300 = xr.concat(filez_all_list[2],dim='region').mean(dim='region')
            ds_bad_v300 = xr.concat(filez_all_list[3],dim='region').mean(dim='region')
            
            ds_good_env.to_netcdf(path+'rawdata/good_forecast_envelope_hovmoller_{}_COMBINED_{}_{}_alt.nc'.format(event,season,model))
            ds_bad_env.to_netcdf(path+'rawdata/bad_forecast_envelope_hovmoller_{}_COMBINED_{}_{}_alt.nc'.format(event,season,model))
            ds_good_v300.to_netcdf(path+'rawdata/good_forecast_v_wind_hovmoller_{}_COMBINED_{}_{}_alt.nc'.format(event,season,model))
            ds_bad_v300.to_netcdf(path+'rawdata/bad_forecast_v_wind_hovmoller_{}_COMBINED_{}_{}_alt.nc'.format(event,season,model))
            
            fig,axz=plt.subplots(2,1,figsize=(16,9))
            ax1 =axz.flat[0];ax2=axz.flat[1]
            lonz = rean_env_good.lon.values
            dayz = rean_env_good.time.values

            clr_levels = np.concatenate((np.arange(-20,-2.4,1.25),np.arange(2.5,21,1.25)))
            contour_levels = contour_dic[season][event]
            im = ax1.contourf(lonz,dayz,ds_good_v300.v.values,cmap='bwr',levels=clr_levels,extend='both')
            ax1.contour(lonz,dayz,ds_good_env.v.values,levels=contour_levels)
            ax1.set_title('Good Forecasts')
            ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
            ax1.axhline(y=0,color='grey')
            ax1.set_ylabel('Days before/after onset event')


            ax2.contourf(lonz,dayz,ds_bad_v300.v.values,cmap='bwr',levels=clr_levels,extend='both')
            ax2.contour(lonz,dayz,ds_bad_env.v.values,levels=contour_levels)

            ax2.set_title('Bad Forecasts')
            ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
            ax2.axhline(y=0,color='grey')
            ax2.set_ylabel('Days before/after onset event')
            ax2.set_xlabel('Longitude [°]')
            fig.suptitle('{} {} {}'.format(event_titlez[i],season,model))

            fig.subplots_adjust(right=0.8)
            cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
            cbar = fig.colorbar(im, cax=cbar_ax)
            cbar.ax.get_yaxis().set_ticks([-20,-15,-10,-5,5,10,15,20])
            cbar.ax.set_yticklabels([-20,-15,-10,-5,5,10,15,20])
            cbar.ax.set_ylabel('v (m/s)')
            
            fig.savefig(path+'good_forecast_envelope_hovmoller_{}_COMBINED_{}_{}_alt.png'.format(event,season,model))
            plt.close(fig)

Combined Hovmöllers for all persistent extremes (envelope only)

In [32]:
coordinatez = [
(54,46,6,14),#Germany
# (44,36,352,360), #Spain
# (54,46,26,34), #Ukraine
(58,50,352,360), #UK
# (42,34,28,36), #Turkey
# (68,60,22,30), #Finland
(66,58,6,14), #Norway/Sweden
# (60,52,46,54), #Russia Samara/Kazan region #Germany
]
path = '/home/onno/Thesis/fcst_composite/hovmoller/reanalysis/'
file = 'rawdata/{}_forecast_{}_hovmoller_{}_lon_{}_{}_lat_{}_{}_{}_{}.nc'
eventz = ['persistent_hw','persistent_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
model_labelz = ['GEFSRF','ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(20,35,1),
                     'persistent_cw': np.arange(20,35,2),
                     'short_hw':np.arange(20,35,1),
                     'short_cw':np.arange(20,35,1)},
               'JJA':{'persistent_hw':np.arange(15,25,1),
                     'persistent_cw': np.arange(15,25,1),
                     'short_hw':np.arange(15,25,1),
                     'short_cw':np.arange(15,25,1)}}
lead_day = 5


ds_good_event_modelz = []
ds_bad_event_modelz = []
for model in modelz:
    ds_good_eventz = []
    ds_bad_eventz = []
    for i,event in enumerate(eventz):
        ds_good_seasonz = []
        ds_bad_seasonz = []
        for season in seasonz:
            ds_good_regionz = []
            ds_bad_regionz = []
            for lat_0,lat_1,lon_0,lon_1 in coordinatez:
                file_good_env = file.format('good','envelope',event,lon_0,lon_1,lat_0,lat_1,season,model)
                file_bad_env = file.format('bad','envelope',event,lon_0,lon_1,lat_0,lat_1,season,model)
                ds_good = xr.open_dataset(path+file_good_env)
                ds_bad = xr.open_dataset(path+file_bad_env)
                ds_good_regionz.append(ds_good)
                ds_bad_regionz.append(ds_bad)
            ds_good_region = xr.concat(ds_good_regionz,dim='region').mean(dim='region')
            ds_bad_region = xr.concat(ds_bad_regionz,dim='region').mean(dim='region')
            ds_good_seasonz.append(ds_good_region)
            ds_bad_seasonz.append(ds_bad_region)
        ds_good_season = xr.concat(ds_good_seasonz,dim='season').mean(dim='season')
        ds_bad_season = xr.concat(ds_bad_seasonz,dim='season').mean(dim='season')
        ds_good_eventz.append(ds_good_season)
        ds_bad_eventz.append(ds_bad_season)
    ds_good_event = xr.concat(ds_good_eventz,dim='event').mean(dim='event')
    ds_bad_event = xr.concat(ds_bad_eventz,dim='event').mean(dim='event')
    ds_good_event.to_netcdf(path+'rawdata/good_forecast_envelope_hovmoller_persistent_ALL_COMBINED_{}.nc'.format(model))
    ds_good_event.to_netcdf(path+'rawdata/bad_forecast_envelope_hovmoller_persistent_ALL_COMBINED_{}.nc'.format(model))
    ds_good_event_modelz.append(ds_good_event)
    ds_bad_event_modelz.append(ds_bad_event)

        



fig,axz=plt.subplots(2,2,figsize=(16,9))
for i in np.arange(len(modelz)):
    ax1 =axz[0,i];ax2=axz[1,i]
    lonz = ds_good_event.lon.values
    dayz = ds_good_event.time.values

    clr_levels_contourf = np.arange(10,25.01,.1)
    clr_levels_contour = np.arange(10,25.01)
    im = ax1.contourf(lonz,dayz,ds_good_event_modelz[i].v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
    cs = ax1.contour(lonz,dayz,ds_good_event_modelz[i].v.values,colors='k',levels=clr_levels_contour,extend='both')
    ax1.clabel(cs,inline=True,fmt='%.0f')

    ax1.set_title(model_labelz[i] + '\n Good Forecasts')
    ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
    ax1.axhline(y=0,color='grey')
    ax1.set_ylabel('Days before/after onset event')
    ax1.set_xlabel('Pseudo Longitude [°]')


    ax2.contourf(lonz,dayz,ds_bad_event_modelz[i].v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
    cs = ax2.contour(lonz,dayz,ds_bad_event_modelz[i].v.values,colors='k',levels=clr_levels_contour,extend='both')
    ax2.clabel(cs,inline=True,fmt='%.0f')

    ax2.set_title('Bad Forecasts')
    ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
    ax2.axhline(y=0,color='grey')
    ax2.set_ylabel('Days before/after onset event')
    ax2.set_xlabel('Pseudo Longitude [°]')
    fig.suptitle('RWP Envelope Composite Analysis All Persistent Extremes Nortwestern Europe')

    fig.subplots_adjust(bottom=0.2,left=0.05,right=0.95,wspace=0.05,hspace=0.15)
    cbar_ax = fig.add_axes([0.1, 0.08, 0.8, 0.045])
    cbar = fig.colorbar(im, cax=cbar_ax,orientation='horizontal')
    cbar.ax.get_xaxis().set_ticks([10,15,20,25])
    cbar.ax.set_xticklabels([10,15,20,25])
    cbar.ax.set_xlabel('E (m/s)')
    ax1.label_outer(); ax2.label_outer()
fig.savefig(path+'combined/persistent_extremes_ALL.png')

#                 fig.savefig(path+'good_forecast_envelope_hovmoller_{}_COMBINED_{}_{}_alt.png'.format(event,season,model))
#                 plt.close(fig)

<ipython-input-32-a98d4626a941>:100: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  cbar_ax = fig.add_axes([0.1, 0.08, 0.8, 0.045])


In [ ]:
coordinatez = [
(54,46,6,14),#Germany
(44,36,352,360), #Spain
(54,46,26,34), #Ukraine
(58,50,352,360), #UK
(42,34,28,36), #Turkey
(68,60,22,30), #Finland
(66,58,6,14), #Norway/Sweden
(60,52,46,54), #Russia Samara/Kazan region #Germany
]

region_labelz = ['DE','ES','UA','UK','TR','FI','NO','RU']
path = '/media/onno/Algemeen/Thesis/fcst_composite/hovmoller/reanalysis/'
file = 'rawdata/{}_forecast_{}_hovmoller_{}_lon_{}_{}_lat_{}_{}_{}_{}.nc'
eventz = ['persistent_hw','persistent_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['DJF']
modelz = ['GFS','ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(20,35,1),
                     'persistent_cw': np.arange(20,35,2),
                     'short_hw':np.arange(20,35,1),
                     'short_cw':np.arange(20,35,1)},
               'JJA':{'persistent_hw':np.arange(15,25,1),
                     'persistent_cw': np.arange(15,25,1),
                     'short_hw':np.arange(15,25,1),
                     'short_cw':np.arange(15,25,1)}}
lead_day = 5

for model in modelz:
    count = 0
    for lat_0,lat_1,lon_0,lon_1 in coordinatez:
        ds_good_eventz = []
        ds_bad_eventz = []
        for i,event in enumerate(eventz):
            ds_good_seasonz = []
            ds_bad_seasonz = []
            for season in seasonz:

                file_good_env = file.format('good','envelope',event,lon_0,lon_1,lat_0,lat_1,season,model)
                file_bad_env = file.format('bad','envelope',event,lon_0,lon_1,lat_0,lat_1,season,model)
                ds_good = xr.open_dataset(path+file_good_env)
                ds_bad = xr.open_dataset(path+file_bad_env)
                ds_good_seasonz.append(ds_good)
                ds_bad_seasonz.append(ds_bad)
            ds_good_season = xr.concat(ds_good_seasonz,dim='season').mean(dim='season')
            ds_bad_season = xr.concat(ds_bad_seasonz,dim='season').mean(dim='season')
            ds_good_eventz.append(ds_good_season)
            ds_bad_eventz.append(ds_bad_season)
        ds_good_event = xr.concat(ds_good_eventz,dim='event').mean(dim='event')
        ds_bad_event = xr.concat(ds_bad_eventz,dim='event').mean(dim='event')
        ds_good_event.to_netcdf(path+'rawdata/good_forecast_envelope_hovmoller_persistent_lon_{}_{}_lat_{}_{}_{}_{}.nc'.format(
        lon_0,lon_1,lat_0,lat_1,model,season))
        ds_good_event.to_netcdf(path+'rawdata/bad_forecast_envelope_hovmoller_persistent_lon_{}_{}_lat_{}_{}_{}_{}.nc'.format(
        lon_0,lon_1,lat_0,lat_1,model,season))






        fig,axz=plt.subplots(2,1,figsize=(16,9))
        ax1 =axz.flat[0];ax2=axz.flat[1]
        lonz = ds_good_event.lon.values
        dayz = ds_good_event.time.values

        clr_levels_contourf = np.arange(10,35.01,.1)
        clr_levels_contour = np.arange(10,35.01,2)
        im = ax1.contourf(lonz,dayz,ds_good_event.v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
        cs = ax1.contour(lonz,dayz,ds_good_event.v.values,colors='k',levels=clr_levels_contour,extend='both')
        ax1.clabel(cs,inline=True,fmt='%.0f')

        ax1.set_title('Good Forecasts')
        ax1.axvline(x=0,color='grey');ax1.axvline(x=0,color='grey')
        ax1.axhline(y=0,color='grey')
        ax1.set_ylabel('Days before/after onset event')


        ax2.contourf(lonz,dayz,ds_bad_event.v.values,cmap=ncl_colormap(),levels=clr_levels_contourf,extend='both')
        cs = ax2.contour(lonz,dayz,ds_bad_event.v.values,colors='k',levels=clr_levels_contour,extend='both')
        ax2.clabel(cs,inline=True,fmt='%.0f')

        ax2.set_title('Bad Forecasts')
        ax2.axvline(x=0,color='grey');ax2.axvline(x=0,color='grey')
        ax2.axhline(y=0,color='grey')
        ax2.set_ylabel('Days before/after onset event')
        ax2.set_xlabel('Longitude [°]')
        fig.suptitle('RWP Envelope Composite Analysis All Persistent Extremes {} {} {}'.format(region_labelz[count],model,season))

        fig.subplots_adjust(right=0.8)
        cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
        cbar = fig.colorbar(im, cax=cbar_ax)
        cbar.ax.get_yaxis().set_ticks([10,15,20,25])
        cbar.ax.set_yticklabels([10,15,20,25])
        cbar.ax.set_ylabel('v (m/s)')
        fig.savefig(path+'combined/persistent_extremes_lon_{}_{}_lat_{}_{}_{}_{}.png'.format(lon_0,lon_1,lat_0,lat_1,model,season))
        plt.close(fig)
        count += 1

#                 fig.savefig(path+'good_forecast_envelope_hovmoller_{}_COMBINED_{}_{}_alt.png'.format(event,season,model))
#                 plt.close(fig)